Пояснение по gen_table: параметр dict_cut выбирает, как обрезать каталоги.

Например при dict_cut = {'z' : [0.1, 0.3]} в каталогах останутся только объекты с 0.1 =< z < 0.3. 

Для b используется модуль: dict_cut = {'b' : [20, np.inf]} оставит в каталогах объекты для которых |b| >= 20 

In [25]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from DS_healpix_fragmentation import radec2pix
from DS_Planck_Unet import val_pix
from DS_data_transformation import calc_error

In [26]:
def get_name(file):
    if 'pz_act' in file:
        return file[5:13]
    return file[5:9]

In [28]:
def cut_cat(df, dict_cut = {'z' : [-np.inf, np.inf], 'M500' : [-np.inf, np.inf], 'b' :[-np.inf, np.inf]}, 
           big_pix=None):
 
    sc = SkyCoord(ra=np.array(df['RA'])*u.degree, 
                  dec=np.array(df['DEC'])*u.degree, frame='icrs')
    df['b'] = sc.galactic.b.degree
    for prm in dict_cut:
        if prm == 'b':
            df = df[np.abs(df[prm]) >= dict_cut[prm][0]]
            df = df[np.abs(df[prm]) < dict_cut[prm][1]]
            df.index = np.arange(len(df))
        else: 
            df = df[df[prm] >= dict_cut[prm][0]]
            df = df[df[prm] < dict_cut[prm][1]]
            df.index = np.arange(len(df))
    
    if not (big_pix is None):
        pix2 = radec2pix(df['RA'], df['DEC'], 2)
        df = df[np.in1d(pix2, big_pix)]
        df.index = np.arange(len(df))
    
    return df

In [29]:
def gen_tables(det_cats_files, true_cats_files, dict_cut = {'z' : [-np.inf, np.inf], 'M500' : [-np.inf, np.inf], 
                        'b' :[-np.inf, np.inf]}, big_pix = None, match_dist=5/60):
    
    true_cats = {os.path.splitext(os.path.basename(file))[0] : pd.read_csv(file) for file in true_cats_files}
    det_cats = {get_name(os.path.splitext(os.path.basename(file))[0]) : 
                pd.read_csv(file) for file in det_cats_files}
    
    comp_df = []
    recall_df = []
    
    for name in det_cats:
        df = det_cats[name]
        df = df[df['status'] != 'fn']
        df.index = np.arange(len(df))
        if 'b' in dict_cut:
            det_cats[name] = cut_cat(df, {'b' : dict_cut['b']}, big_pix)
        else:
            det_cats[name] = cut_cat(df, {}, big_pix)
    for name in true_cats:
        true_cats[name] = cut_cat(true_cats[name], dict_cut, big_pix)
    
    for det_name in det_cats:
        det = det_cats[det_name]
        line = {}
        line_r = {}

        det_sc = SkyCoord(ra=np.array(det['RA'])*u.degree, 
                      dec=np.array(det['DEC'])*u.degree, frame='icrs') 

        for tr_name in true_cats: 
            tr = true_cats[tr_name]
            tr_sc = SkyCoord(ra=np.array(tr['RA'])*u.degree, 
                          dec=np.array(tr['DEC'])*u.degree, frame='icrs')
            
            idx, d2d, _ = tr_sc.match_to_catalog_sky(det_sc)
            matched = d2d.degree <= match_dist
            
            line[tr_name] = np.count_nonzero(matched)
            line[tr_name+'_err'], line[tr_name+'_std'] = calc_error(det, tr)

            line_r[tr_name] = line[tr_name] / len(tr)
            
        line['all'] = len(det)
        line['fp'] = np.count_nonzero(det['status'] == 'fp')
        line_r['fp'] = line['fp']
        line_r['all'] = line['all']
        comp_df.append(pd.DataFrame(line, index=[det_name]))
        recall_df.append(pd.DataFrame(line_r, index=[det_name]))
        line = {}
    
    for tr_name in true_cats: 
        line[tr_name] = len(true_cats[tr_name])
        line[tr_name+'_err'] = 0
    line['fp'] = 0
    line['all'] = 0
    comp_df.append(pd.DataFrame(line, index=['all']))
    
    comp_df = pd.concat(comp_df)
    recall_df = pd.concat(recall_df)
    
    return comp_df, recall_df

In [22]:
tr_dir = '/home/rt2122/Data/original_catalogs/csv/'
true_cats_files = [os.path.join(tr_dir, name) for name in next(os.walk(tr_dir))[-1]]
det_dir = '/home/rt2122/Data/detected_cats/'
det_cats_files = [os.path.join(det_dir, name) for name in sorted(next(os.walk(det_dir))[-1]) if 'full' in name]
#det_cats_files = ['/home/rt2122/Data/detected_cats/full_pz14_thr0.1_step8.csv', 
#                 '/home/rt2122/Data/detected_cats/full_pz_act10_thr0.1_step8.csv']

In [30]:
comp, recall = gen_tables(det_cats_files, true_cats_files, dict_cut={'b' : [20, np.inf]})
comp

<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,RM,RM_err,RM_std,ACT,ACT_err,ACT_std,all,fp
pz14,1226,11.25,0.664415,647,14.45,0.737974,1268,285.35,4.666947,858,43.45,1.164101,12823,10971
pz20,1257,12.25,0.706641,661,16.40,0.758808,1342,304.75,4.844897,887,45.50,1.792528,14271,12355
pz25,1246,14.40,1.108342,665,17.75,0.570664,1415,321.50,3.571451,895,50.30,1.643328,15388,13468
pz40,1235,14.15,0.805491,659,18.15,0.625237,1305,314.95,2.874090,881,48.90,1.325658,14127,12233
pz_act06,1034,4.95,0.450000,527,6.30,0.589826,750,118.05,2.588919,614,17.75,1.035870,5414,4031
pz_act10,1134,8.65,0.595045,599,10.60,0.685949,1051,206.55,3.583937,940,30.15,0.859238,8587,6780
pz_act14,1146,9.85,0.689298,608,13.65,0.949446,1245,250.45,2.855719,1301,37.65,1.086460,10691,8529
pz_act20,1047,7.25,0.672290,546,9.00,0.593828,1221,184.55,3.878738,1884,29.70,1.167769,8122,5463
pz_act25,1068,8.20,0.495241,548,10.45,0.789653,1312,206.50,2.576718,2169,35.60,1.098803,9636,6675
all,1342,0.00,NaN,1612,0.00,NaN,25999,0.00,NaN,4195,0.00,NaN,0,0


In [31]:
recall

,PSZ2,MCXC,RM,ACT,fp,all
pz14,0.913562,0.401365,0.048771,0.204529,10971,12823
pz20,0.936662,0.410050,0.051617,0.211442,12355,14271
pz25,0.928465,0.412531,0.054425,0.213349,13468,15388
pz40,0.920268,0.408809,0.050194,0.210012,12233,14127
pz_act06,0.770492,0.326923,0.028847,0.146365,4031,5414
pz_act10,0.845007,0.371588,0.040425,0.224076,6780,8587
pz_act14,0.853949,0.377171,0.047886,0.310131,8529,10691
pz_act20,0.780179,0.338710,0.046963,0.449106,5463,8122
pz_act25,0.795827,0.339950,0.050463,0.517044,6675,9636


In [32]:
true_cats_files = [file for file in true_cats_files if not ('RM' in file)]

In [33]:
comp, recall = gen_tables(det_cats_files, true_cats_files, dict_cut={'M500' : [3, np.inf], 'b' : [20, np.inf]})
comp

<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,753,6.35,0.553815,309,2.90,0.463965,465,9.20,0.573998,12823,10971
pz20,767,6.95,0.638563,306,4.30,0.348682,462,9.85,0.693105,14271,12355
pz25,767,8.75,0.797612,303,3.70,0.470722,474,11.05,0.609033,15388,13468
pz40,759,9.20,0.651314,304,4.65,0.405716,466,10.35,0.567891,14127,12233
pz_act06,667,3.00,0.307794,276,1.40,0.275299,354,3.25,0.260314,5414,4031
pz_act10,709,5.05,0.455810,295,2.55,0.343932,470,6.25,0.485988,8587,6780
pz_act14,719,6.05,0.499868,298,3.75,0.501970,549,7.45,0.520501,10691,8529
pz_act20,685,5.00,0.435286,285,2.15,0.310136,645,6.75,0.551911,8122,5463
pz_act25,693,5.10,0.376270,284,2.50,0.351688,692,7.50,0.505236,9636,6675
all,775,0.00,NaN,378,0.00,NaN,975,0.00,NaN,0,0


In [34]:
recall

,PSZ2,MCXC,ACT,fp,all
pz14,0.971613,0.817460,0.476923,10971,12823
pz20,0.989677,0.809524,0.473846,12355,14271
pz25,0.989677,0.801587,0.486154,13468,15388
pz40,0.979355,0.804233,0.477949,12233,14127
pz_act06,0.860645,0.730159,0.363077,4031,5414
pz_act10,0.914839,0.780423,0.482051,6780,8587
pz_act14,0.927742,0.788360,0.563077,8529,10691
pz_act20,0.883871,0.753968,0.661538,5463,8122
pz_act25,0.894194,0.751323,0.709744,6675,9636


In [35]:
comp, recall = gen_tables(det_cats_files, true_cats_files, dict_cut={'M500' : [4, np.inf], 'b' : [20, np.inf]})
comp

<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,605,5.15,0.460406,199,1.80,0.337171,258,3.40,0.466792,12823,10971
pz20,613,5.45,0.630267,197,2.55,0.343932,257,3.15,0.466087,14271,12355
pz25,612,7.30,0.703749,195,2.40,0.350939,265,4.05,0.413426,15388,13468
pz40,608,7.25,0.631935,197,2.50,0.400657,263,4.45,0.400493,14127,12233
pz_act06,543,2.70,0.281911,187,0.80,0.171679,213,1.00,0.205196,5414,4031
pz_act10,575,3.90,0.463965,196,1.50,0.294690,248,2.30,0.308647,8587,6780
pz_act14,581,5.20,0.540954,197,2.55,0.444114,266,2.85,0.424729,10691,8529
pz_act20,556,4.00,0.423022,193,1.60,0.275299,274,2.35,0.334625,8122,5463
pz_act25,564,4.40,0.425503,195,1.50,0.312039,282,2.20,0.352435,9636,6675
all,619,0.00,NaN,232,0.00,NaN,342,0.00,NaN,0,0


In [36]:
recall

,PSZ2,MCXC,ACT,fp,all
pz14,0.977383,0.857759,0.754386,10971,12823
pz20,0.990307,0.849138,0.751462,12355,14271
pz25,0.988691,0.840517,0.774854,13468,15388
pz40,0.982229,0.849138,0.769006,12233,14127
pz_act06,0.877221,0.806034,0.622807,4031,5414
pz_act10,0.928918,0.844828,0.725146,6780,8587
pz_act14,0.938611,0.849138,0.777778,8529,10691
pz_act20,0.898223,0.831897,0.801170,5463,8122
pz_act25,0.911147,0.840517,0.824561,6675,9636


In [37]:
comp, recall = gen_tables(det_cats_files, true_cats_files, dict_cut={'M500' : [5, np.inf], 'b' : [20, np.inf]})
comp

<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,411,3.65,0.385766,107,0.95,0.169752,140,1.20,0.247088,12823,10971
pz20,413,3.25,0.440544,106,1.45,0.245753,141,1.55,0.234801,14271,12355
pz25,413,4.75,0.463823,105,1.10,0.239517,142,1.75,0.270234,15388,13468
pz40,410,4.35,0.514398,107,1.05,0.285390,141,2.00,0.271448,14127,12233
pz_act06,381,1.90,0.250263,102,0.30,0.105131,119,0.50,0.135724,5414,4031
pz_act10,397,2.80,0.407818,107,0.80,0.171679,132,1.25,0.190221,8587,6780
pz_act14,401,3.70,0.497890,106,1.20,0.304354,135,1.25,0.227977,10691,8529
pz_act20,385,2.40,0.327671,104,0.75,0.190221,134,0.90,0.228266,8122,5463
pz_act25,392,2.90,0.250263,105,0.80,0.212751,137,0.90,0.190567,9636,6675
all,417,0.00,NaN,121,0.00,NaN,153,0.00,NaN,0,0


In [38]:
recall

,PSZ2,MCXC,ACT,fp,all
pz14,0.985612,0.884298,0.915033,10971,12823
pz20,0.990408,0.876033,0.921569,12355,14271
pz25,0.990408,0.867769,0.928105,13468,15388
pz40,0.983213,0.884298,0.921569,12233,14127
pz_act06,0.913669,0.842975,0.777778,4031,5414
pz_act10,0.952038,0.884298,0.862745,6780,8587
pz_act14,0.961631,0.876033,0.882353,8529,10691
pz_act20,0.923261,0.859504,0.875817,5463,8122
pz_act25,0.940048,0.867769,0.895425,6675,9636


In [39]:
comp, recall = gen_tables(det_cats_files, true_cats_files, dict_cut={'b' : [20, np.inf]})
comp

<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,1226,11.25,0.664415,647,14.45,0.737974,858,43.45,1.164101,12823,10971
pz20,1257,12.25,0.706641,661,16.40,0.758808,887,45.50,1.792528,14271,12355
pz25,1246,14.40,1.108342,665,17.75,0.570664,895,50.30,1.643328,15388,13468
pz40,1235,14.15,0.805491,659,18.15,0.625237,881,48.90,1.325658,14127,12233
pz_act06,1034,4.95,0.450000,527,6.30,0.589826,614,17.75,1.035870,5414,4031
pz_act10,1134,8.65,0.595045,599,10.60,0.685949,940,30.15,0.859238,8587,6780
pz_act14,1146,9.85,0.689298,608,13.65,0.949446,1301,37.65,1.086460,10691,8529
pz_act20,1047,7.25,0.672290,546,9.00,0.593828,1884,29.70,1.167769,8122,5463
pz_act25,1068,8.20,0.495241,548,10.45,0.789653,2169,35.60,1.098803,9636,6675
all,1342,0.00,NaN,1612,0.00,NaN,4195,0.00,NaN,0,0


In [40]:
recall

,PSZ2,MCXC,ACT,fp,all
pz14,0.913562,0.401365,0.204529,10971,12823
pz20,0.936662,0.410050,0.211442,12355,14271
pz25,0.928465,0.412531,0.213349,13468,15388
pz40,0.920268,0.408809,0.210012,12233,14127
pz_act06,0.770492,0.326923,0.146365,4031,5414
pz_act10,0.845007,0.371588,0.224076,6780,8587
pz_act14,0.853949,0.377171,0.310131,8529,10691
pz_act20,0.780179,0.338710,0.449106,5463,8122
pz_act25,0.795827,0.339950,0.517044,6675,9636


In [41]:
comp, recall = gen_tables(det_cats_files, true_cats_files, dict_cut={'M500' : [3, np.inf],
                                                                    'z' : [-np.inf, 0.1], 
                                                                     'b' : [20, np.inf]})
comp

<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,47,0.60,0.183533,35,0.45,0.169752,13,0.25,0.099340,12823,10971
pz20,48,0.40,0.152177,37,0.25,0.099340,13,0.10,0.068825,14271,12355
pz25,50,0.60,0.222427,37,0.40,0.152177,14,0.10,0.068825,15388,13468
pz40,48,0.50,0.153897,35,0.50,0.184961,14,0.20,0.091766,14127,12233
pz_act06,49,0.15,0.081918,36,0.30,0.127733,13,0.10,0.068825,5414,4031
pz_act10,49,0.40,0.112390,37,0.25,0.099340,14,0.15,0.081918,8587,6780
pz_act14,49,0.40,0.133771,37,0.25,0.099340,14,0.05,0.050000,10691,8529
pz_act20,49,0.65,0.150000,39,0.50,0.135724,15,0.25,0.123010,8122,5463
pz_act25,49,0.45,0.153469,38,0.50,0.223607,15,0.25,0.123010,9636,6675
all,52,0.00,NaN,40,0.00,NaN,17,0.00,NaN,0,0


In [42]:
recall

,PSZ2,MCXC,ACT,fp,all
pz14,0.903846,0.875,0.764706,10971,12823
pz20,0.923077,0.925,0.764706,12355,14271
pz25,0.961538,0.925,0.823529,13468,15388
pz40,0.923077,0.875,0.823529,12233,14127
pz_act06,0.942308,0.900,0.764706,4031,5414
pz_act10,0.942308,0.925,0.823529,6780,8587
pz_act14,0.942308,0.925,0.823529,8529,10691
pz_act20,0.942308,0.975,0.882353,5463,8122
pz_act25,0.942308,0.950,0.882353,6675,9636


In [43]:
comp, recall = gen_tables(det_cats_files, true_cats_files, dict_cut={'M500' : [3, np.inf],
                                                                    'z' : [0.1, 0.3], 'b' : [20, np.inf]})
comp

<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,386,2.85,0.482728,203,1.70,0.325253,132,1.10,0.216430,12823,10971
pz20,393,3.55,0.483817,198,2.90,0.362012,135,1.40,0.319539,14271,12355
pz25,391,3.90,0.422399,196,2.20,0.312881,137,1.15,0.264326,15388,13468
pz40,389,5.15,0.524530,198,3.20,0.321182,133,1.70,0.281911,14127,12233
pz_act06,351,1.65,0.264326,181,0.75,0.216126,120,0.35,0.131289,5414,4031
pz_act10,367,2.60,0.284697,191,1.75,0.279803,131,0.95,0.198348,8587,6780
pz_act14,371,2.60,0.386618,193,2.45,0.387128,137,1.25,0.279803,10691,8529
pz_act20,358,2.45,0.285390,186,1.20,0.257519,134,0.75,0.160181,8122,5463
pz_act25,358,2.70,0.333246,183,1.50,0.211511,135,1.15,0.220943,9636,6675
all,394,0.00,NaN,246,0.00,NaN,159,0.00,NaN,0,0


In [44]:
recall

,PSZ2,MCXC,ACT,fp,all
pz14,0.979695,0.825203,0.830189,10971,12823
pz20,0.997462,0.804878,0.849057,12355,14271
pz25,0.992386,0.796748,0.861635,13468,15388
pz40,0.987310,0.804878,0.836478,12233,14127
pz_act06,0.890863,0.735772,0.754717,4031,5414
pz_act10,0.931472,0.776423,0.823899,6780,8587
pz_act14,0.941624,0.784553,0.861635,8529,10691
pz_act20,0.908629,0.756098,0.842767,5463,8122
pz_act25,0.908629,0.743902,0.849057,6675,9636


In [45]:
comp, recall = gen_tables(det_cats_files, true_cats_files, dict_cut={'M500' : [3, np.inf],
                                                                    'z' : [0.3, 0.6], 'b' : [20, np.inf]})
comp

<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,291,2.75,0.298020,65,0.65,0.150000,239,4.60,0.293795,12823,10971
pz20,295,2.85,0.364728,65,1.10,0.280038,239,4.75,0.475035,14271,12355
pz25,295,3.90,0.396697,64,1.05,0.223312,241,5.95,0.587031,15388,13468
pz40,292,3.25,0.389838,65,0.85,0.208693,233,4.50,0.328473,14127,12233
pz_act06,245,1.10,0.239517,57,0.35,0.131289,174,1.75,0.239242,5414,4031
pz_act10,267,1.95,0.311828,63,0.50,0.184961,230,2.75,0.331464,8587,6780
pz_act14,272,2.95,0.358909,63,0.95,0.245753,264,3.50,0.461690,10691,8529
pz_act20,254,1.80,0.381134,58,0.40,0.133771,303,3.05,0.303272,8122,5463
pz_act25,261,1.80,0.224781,60,0.50,0.153897,326,3.60,0.327671,9636,6675
all,298,0.00,NaN,85,0.00,NaN,457,0.00,NaN,0,0


In [46]:
recall

,PSZ2,MCXC,ACT,fp,all
pz14,0.976510,0.764706,0.522976,10971,12823
pz20,0.989933,0.764706,0.522976,12355,14271
pz25,0.989933,0.752941,0.527352,13468,15388
pz40,0.979866,0.764706,0.509847,12233,14127
pz_act06,0.822148,0.670588,0.380744,4031,5414
pz_act10,0.895973,0.741176,0.503282,6780,8587
pz_act14,0.912752,0.741176,0.577681,8529,10691
pz_act20,0.852349,0.682353,0.663020,5463,8122
pz_act25,0.875839,0.705882,0.713348,6675,9636


In [47]:
comp, recall = gen_tables(det_cats_files, true_cats_files, dict_cut={'M500' : [3, np.inf],
                                                                    'z' : [0.6, np.inf], 
                                                                     'b' : [20, np.inf]})
comp

<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,29,0.15,0.081918,6,0.10,0.068825,81,3.25,0.458114,12823,10971
pz20,31,0.15,0.081918,6,0.05,0.050000,75,3.60,0.379751,14271,12355
pz25,31,0.35,0.150000,6,0.05,0.050000,82,3.85,0.442927,15388,13468
pz40,30,0.30,0.127733,6,0.10,0.068825,86,3.95,0.489226,14127,12233
pz_act06,22,0.10,0.068825,2,0.00,0.000000,47,1.05,0.234801,5414,4031
pz_act10,26,0.10,0.068825,4,0.05,0.050000,95,2.40,0.293795,8587,6780
pz_act14,27,0.10,0.068825,5,0.10,0.068825,134,2.65,0.364728,10691,8529
pz_act20,24,0.10,0.100000,2,0.05,0.050000,193,2.70,0.356149,8122,5463
pz_act25,25,0.15,0.081918,3,0.00,0.000000,216,2.50,0.336389,9636,6675
all,31,0.00,NaN,7,0.00,NaN,342,0.00,NaN,0,0


In [48]:
recall

,PSZ2,MCXC,ACT,fp,all
pz14,0.935484,0.857143,0.236842,10971,12823
pz20,1.000000,0.857143,0.219298,12355,14271
pz25,1.000000,0.857143,0.239766,13468,15388
pz40,0.967742,0.857143,0.251462,12233,14127
pz_act06,0.709677,0.285714,0.137427,4031,5414
pz_act10,0.838710,0.571429,0.277778,6780,8587
pz_act14,0.870968,0.714286,0.391813,8529,10691
pz_act20,0.774194,0.285714,0.564327,5463,8122
pz_act25,0.806452,0.428571,0.631579,6675,9636


In [49]:
comp, recall = gen_tables(det_cats_files, true_cats_files, {}, val_pix)
comp

<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree
<ipython-input-28-414464267a46>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


,PSZ2,PSZ2_err,PSZ2_std,MCXC,MCXC_err,MCXC_std,ACT,ACT_err,ACT_std,all,fp
pz14,148,0.80,0.200000,72,0.85,0.232549,127,2.20,0.267542,1281,1055
pz20,151,1.05,0.211200,72,0.75,0.227977,147,2.65,0.334625,1361,1116
pz25,150,0.85,0.232549,74,0.90,0.160591,148,3.40,0.494177,1436,1187
pz40,147,0.70,0.179179,70,0.70,0.193309,132,3.20,0.587591,1409,1180
pz_act06,123,0.15,0.081918,56,0.20,0.091766,69,1.10,0.190567,524,375
pz_act10,144,0.85,0.166623,65,0.60,0.168585,96,1.85,0.274101,814,624
pz_act14,143,0.85,0.181731,63,0.40,0.133771,96,2.10,0.289282,947,759
pz_act20,133,0.20,0.091766,59,0.25,0.099340,73,1.35,0.220943,716,557
pz_act25,132,0.45,0.153469,59,0.45,0.135239,78,1.95,0.328273,787,624
all,160,0.00,NaN,166,0.00,NaN,602,0.00,NaN,0,0


In [50]:
recall

,PSZ2,MCXC,ACT,fp,all
pz14,0.92500,0.433735,0.210963,1055,1281
pz20,0.94375,0.433735,0.244186,1116,1361
pz25,0.93750,0.445783,0.245847,1187,1436
pz40,0.91875,0.421687,0.219269,1180,1409
pz_act06,0.76875,0.337349,0.114618,375,524
pz_act10,0.90000,0.391566,0.159468,624,814
pz_act14,0.89375,0.379518,0.159468,759,947
pz_act20,0.83125,0.355422,0.121262,557,716
pz_act25,0.82500,0.355422,0.129568,624,787
